**Job listings Database cleaning project**

In [1]:
import numpy as np
import pandas as pd
#indian_df = pd.read_csv('indian_database.csv')
india_df = pd.read_csv('job_cleanData.csv')
canada_df = pd.read_csv('linkedin-jobs-canada.csv')
usa_df = pd.read_csv('linkedin-jobs-usa.csv')

In [2]:
#This segment drops non-relevant columns for future analysis.
#Additionally, I also renamed some columns in order to have them with the same name as the other dataframes (Canada and USA).
#Second to last line is used to identify the data from this df once all are concatenated. Each df row will have its country info.

india_df = india_df.drop(columns=['City', 'job_ID', 'company_id', 'involvement', 'employees_count', 'linkedin_followers', 'details_id', 'State', 'total_applicants'])
india_df.rename(columns={'designation':'title', 'name':'company', 'work_type':'onsite_remote', 'job_details':'criteria'}, inplace=True)
india_df['country'] = 'India'
print(india_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5587 entries, 0 to 5586
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   title          5587 non-null   object
 1   company        5587 non-null   object
 2   onsite_remote  5587 non-null   object
 3   criteria       5587 non-null   object
 4   industry       5587 non-null   object
 5   level          5587 non-null   object
 6   country        5587 non-null   object
dtypes: object(7)
memory usage: 305.7+ KB
None


In [3]:
#This section is doing pretty much the same as the last one.
#I dropped non-relevant columns and added the country tag to all rows for concatenation. 

canada_df = canada_df.drop(columns=['description', 'salary', 'link', 'location'])
canada_df['country'] = 'Canada'
canada_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2773 entries, 0 to 2772
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   title          2773 non-null   object
 1   company        2773 non-null   object
 2   onsite_remote  2773 non-null   object
 3   criteria       2773 non-null   object
 4   posted_date    2773 non-null   object
 5   country        2773 non-null   object
dtypes: object(6)
memory usage: 130.1+ KB


In [4]:
#This section is doing pretty much the same as the last one.
#I dropped non-relevant columns and added the country tag to all rows for concatenation. 

usa_df = usa_df.drop(columns=['salary', 'location', 'description', 'link'])
usa_df['country'] = 'USA'
usa_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2845 entries, 0 to 2844
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   title          2845 non-null   object
 1   company        2845 non-null   object
 2   onsite_remote  2845 non-null   object
 3   criteria       2845 non-null   object
 4   posted_date    2845 non-null   object
 5   country        2845 non-null   object
dtypes: object(6)
memory usage: 133.5+ KB


In [5]:
#This part concatenates all dfs
#Further columns were dropped after realising lack of relevance or way too much nulls. 
#posted_date variable type updated to pandas datetime + filling nulls with standard date '2022-01-01', nulls were indeed identified as data from 2022. No day or month was found.

complete_df = pd.concat([india_df, canada_df, usa_df])
complete_df.drop(columns=['industry'])
complete_df['posted_date'] = pd.to_datetime(complete_df['posted_date'], format='%Y-%m-%d', errors='coerce')
complete_df['posted_date'] = complete_df['posted_date'].fillna(pd.Timestamp('2022-01-01'))
complete_df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 11205 entries, 0 to 2844
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   title          11205 non-null  object        
 1   company        11205 non-null  object        
 2   onsite_remote  11205 non-null  object        
 3   criteria       11205 non-null  object        
 4   industry       5587 non-null   object        
 5   level          5587 non-null   object        
 6   country        11205 non-null  object        
 7   posted_date    11205 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(7)
memory usage: 787.9+ KB


In [6]:
#Identify unique values for 'title' column
#clean and standardize names with .lower(), .strip(), and regex.

complete_df['title_cleaned'] = (
    complete_df['title']
    .str.lower()
    .str.strip()
    .str.replace(r'\b(senior|junior|lead|remote|onsite|hybrid|freelance|intern|contract|recent|graduate)\b', '', regex=True)
    .str.replace(r'\(\s*\)', '', regex=True)
    .str.replace(r'[-–—]', ' ', regex=True)
    .str.replace(r'[^\w\s]', '', regex=True)
    .str.replace(r'\s+', ' ', regex=True)
    .str.strip()
)

print(complete_df.title_cleaned.nunique())
titles_df = complete_df['title_cleaned'].value_counts()
complete_df['title'] = complete_df['title_cleaned']
complete_df.drop(columns=['title_cleaned'], inplace=True)

199


In [7]:
#Here I used AI(Copilot) to determine the best semantic mapping in order to standardize title names.
#I pasted all unique names on Copilot and asked to create a semantic map. Here is the result.
#I checked for un-categorized titles and updated the map in order to categorize the remaining ones. 

title_map = {
    # Data Analyst Cluster
    'data analyst': 'data analyst',
    'data analytics': 'data analyst',
    'analyst': 'data analyst',
    'data analytics analyst': 'data analyst',
    'data analystdeveloper': 'data analyst',
    'data analyst operations': 'data analyst',
    'data analyst loans': 'data analyst',
    'data analystoracle': 'data analyst',
    'data analyst marketing': 'data analyst',
    'data analyst apac marketplace': 'data analyst',
    'data analyst toronto on': 'data analyst',
    'data analyst trilogy 60000year usd': 'data analyst',
    'data analyst analytics insights toronto on': 'data analyst',
    'data analyst with guidewire experience us canada': 'data analyst',
    'data analyst vancouver bc': 'data analyst',
    'data analystforecasting specialist': 'data analyst',
    'data analyst c117': 'data analyst',
    'data analyst 12 month': 'data analyst',
    'data analyst pythonsql': 'data analyst',
    'data analyst reporting': 'data analyst',
    'data analyst flight pricing optimization': 'data analyst',
    'sector data analyst data driven investing': 'data analyst',
    'data administratordata analyst': 'data analyst',
    'commercial data analyst': 'data analyst',
    'data analyst powerbi': 'data analyst',
    'data analyst land': 'data analyst',
    'operational data analyst': 'data analyst',
    'campaigndata analyst': 'data analyst',
    'entry level data analyst': 'data analyst',
    'data analyst money': 'data analyst',
    'data analystcollector': 'data analyst',
    'data analyst i': 'data analyst',
    'data analyst i entry level': 'data analyst',
    'data analyst us': 'data analyst',
    'data analyst sql operations': 'data analyst',
    'data analyst sql': 'data analyst',
    'data analyst global': 'data analyst',
    'wfhdata analyst': 'data analyst',
    'data analyst iii': 'data analyst',
    'data analyst sql teraform tableau iii': 'data analyst',
    'associate data analyst': 'data analyst',
    'data analyst 100m valuation': 'data analyst',
    'data analyst weekly schedule': 'data analyst',
    'group data analyst': 'data analyst',
    'consultantdata analyst': 'data analyst',
    'data analyst energy': 'data analyst',
    'data analyst entry level': 'data analyst',
    'sr data analyst': 'data analyst',
    'hr data analyst': 'data analyst',
    'quality data analyst': 'data analyst',
    'data entry analyst risk': 'data analyst',
    'data entry jr analyst 6 month': 'data analyst',
    'market conduct data analyst 4 month co op': 'data analyst',
    'sr market data analyst': 'data analyst',
    'principal data analyst': 'data analyst',
    'data analyst loanscapital markets': 'data analyst',
    'data analyst consultant fire ems': 'data analyst',
    'future opportunities data analyst report specialist': 'data analyst',

    # Data Scientist & ML Cluster
    'data scientist': 'data scientist',
    'data science analyst': 'data scientist',
    'analytics engineer': 'data scientist',
    'analyst data science en': 'data scientist',
    'analyst data science fr': 'data scientist',
    'analyst global data and analytics': 'data scientist',
    'analyst data and analytics': 'data scientist',
    'analyst data visualization': 'data scientist',
    'analyst cyber governance data analytics': 'data scientist',
    'data visualization analyst': 'data scientist',
    'data visualization developeranalyst': 'data scientist',
    'gaming data analyst': 'data scientist',
    'cognitive data analyst': 'data scientist',
    'google cloud platform analytics engineer': 'data scientist',
    'google analytics engineer': 'data scientist',
    'marketing data analyst': 'data scientist',
    'seo data analyst': 'data scientist',

    # Data Engineer Cluster
    'data engineer': 'data engineer',
    'data engineer full time': 'data engineer',
    'data engineerbig data engineer': 'data engineer',
    'data engineer totogi 60000year usd': 'data engineer',
    'data engineer ii': 'data engineer',
    'data engineer 1': 'data engineer',
    'data engineer azure': 'data engineer',
    'data engineer with ai ml analytics platforms': 'data engineer',
    'data engineer data aws etl': 'data engineer',

    # BI & Reporting Cluster
    'business intelligence analyst': 'bi analyst',
    'bi analyst': 'bi analyst',
    'bi reporting analyst': 'bi analyst',
    'business intelligence analyst fpa': 'bi analyst',
    'power bi developer': 'bi analyst',
    'powerbi specialist': 'bi analyst',

    # Other Analyst Roles
    'information analyst': 'analyst',
    'information governance analyst': 'analyst',
    'digital analyst': 'analyst',
    'insights analyst': 'analyst',
    'data specialist': 'analyst',
    'data management': 'analyst',
    'quantitative data': 'analyst',
    'quantitative trader': 'analyst',
    'geospatial analyst available 2023': 'analyst',

    # Business Analyst Cluster
    'business analyst': 'business analyst',
    'business systems analyst': 'business analyst',
    'business data analyst': 'business analyst',
    'business data analyst finance': 'business analyst',

    # Developer Cluster
    'developer': 'developer',
    'shopify developer': 'developer',
    'database developer': 'developer',
    'salesforce developer': 'developer',
    'golang developer': 'developer',
    'net developers': 'developer',
    'php developer': 'developer',
    'sql developer': 'developer',
    'java developer': 'developer',
    'python developer': 'developer',
    'powerapps developer': 'developer',
    'snowflake developer': 'developer',
    'angular developer': 'developer',
    'oracle developer': 'developer',
    'blockchain developer': 'developer',
    'c developer': 'developer',
    'ruby on rails developer': 'developer',
    'game developer': 'developer',
    'mainframe developer': 'developer',
    'unity developer': 'developer',
    'mobile application developer': 'developer',
    'full stack developer': 'developer',
    'web developer': 'developer',
    'drupal developer': 'developer',
    'wordpress developer': 'developer',
    'backend developer': 'developer',
    'node js developer': 'developer',
    'sap developer': 'developer',
    'vuejs developer': 'developer',
    'reactjs developer': 'developer',
    'bigdata developer': 'developer',
    'job opportunity data stage developer': 'developer',

    # AI & Robotics Cluster
    'ai engineer': 'ai engineer',
    'robotics': 'ai engineer',

    # QA & Testing
    'software testing': 'qa engineer',
    'quality analyst': 'qa engineer',

    # Education & Training
    'technical trainerwriterowner': 'trainer',
    'professor': 'trainer',
    'education and counseling': 'trainer',

    # Management & Leadership
    'managerial and leadership roles': 'manager',
    'project manager': 'manager',
    'scrum master': 'manager',
    'team project': 'manager',
    'head of product': 'manager',
    'product specialist': 'manager',

    # Finance & Accounting
    'accountant': 'finance',
    'finance manager': 'finance',
    'financial controller': 'finance',

    # Marketing & Content
    'marketing': 'marketing',
    'digital marketing': 'marketing',
    'copywriter': 'marketing',
    'content writer': 'marketing',
    'search engine optimization': 'marketing',

    # Sales & Customer Service
    'sales executive': 'sales',
    'business development manager': 'sales',
    'salesperson': 'sales',
    'relationship manager': 'sales',
    'customer service': 'sales',
    'collections specialist': 'sales',

    # Miscellaneous
    'consultant': 'consultant',
    'associate': 'associate',
    'researcher': 'researcher',
    'co founder': 'executive',
    'storage administrator': 'it support',
    'technical support analyst': 'it support',
    'technology architecture': 'it support',
    'online data analyst': 'data analyst',
    'bianalytics consultant': 'bi analyst',
}
title_map.update({
    # Data Analyst variants
    'data analyst mississauga on': 'data analyst',
    'sql data analyst': 'data analyst',
    'product data analyst wtfast': 'data analyst',
    'product data analyst': 'data analyst',
    'data research analyst trilogy 60000year usd': 'data analyst',
    'data and reporting analyst': 'data analyst',
    'data governance analyst': 'data analyst',
    'cybersecurity data analyst': 'data analyst',
    'analyste de données': 'data analyst',
    'it data analyst sql banking finance': 'data analyst',
    'data analysts': 'data analyst',
    'cloud data analyst': 'data analyst',
    'analyst data operations': 'data analyst',
    'data analyst 2': 'data analyst',

    # BI Analyst
    'bi analyst': 'bi analyst',
    'analyste bi': 'bi analyst',

    # Developer variants
    'front end developer': 'developer',
    'frontend developer': 'developer',
    'ios developer': 'developer',

    # AI & Engineering
    'ai engineer': 'ai engineer',
    'linux engineer': 'engineer',

    # Research & Support
    'researcher': 'researcher',
    'associate': 'associate',
    'executive': 'executive',
    'editor': 'marketing',  # or 'content' if you prefer a separate cluster
})


In [8]:
#Mapping used to change 'title' column for new names and standarization 
complete_df['title'] = complete_df['title'].map(title_map).fillna(complete_df['title'])
print(complete_df.title.nunique())
print(complete_df['title'].value_counts())

28
title
data analyst                 6123
developer                    1930
qa engineer                   444
other developer               395
sales                         323
business analyst              274
other                         230
data scientist                205
manager                       192
other engineering             151
internships                   130
marketing                     121
consultant                     84
cloud engineer                 72
software engineer              68
finance                        63
analyst                        59
it support                     57
devops engineer                49
machine learning engineer      42
trainer                        41
data engineer                  39
bi analyst                     37
ai engineer                    29
associate                      26
engineer                       11
researcher                      6
executive                       4
Name: count, dtype: int64


In [9]:
#Here I standardized the company name column in order to have all of them in lowercase
complete_df['company'] = complete_df['company'].str.lower().str.strip()

In [10]:
#I used this part to standardize capitalizaiton and spelling on 'onsite_remote' column.
complete_df['onsite_remote'] = complete_df['onsite_remote'].str.lower().str.strip().str.replace('-','')
print(complete_df['onsite_remote'].value_counts())

onsite_remote
onsite    4259
remote    4133
hybrid    2813
Name: count, dtype: int64


AI Segment (Copilot): Code used to filter and extract common words/keywords from the 'criteria' description column.

After reviewing the top 50 words, the decision to drop the column mas made. But instead of dropping the column, a new dataframe without the 'criteria' column was made. Just in case any future NLP modeling is performed. 

In [11]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

# Fill missing values and limit to top 1000 words
vectorizer = CountVectorizer(stop_words='english', max_features=1000)

# Fit and transform the criteria column
X = vectorizer.fit_transform(complete_df['criteria'].fillna(''))

# Create a dictionary of word frequencies
word_freq = dict(zip(vectorizer.get_feature_names_out(), X.sum(axis=0).A1))

# Sort and print top 50 words
sorted_words = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)
for word, freq in sorted_words[:50]:
    print(f"{word}: {freq}")


experience: 19375
job: 17708
epam: 13760
work: 10852
data: 10756
level: 10284
technology: 8744
development: 8089
global: 7981
skills: 7779
years: 7624
team: 7458
services: 7247
software: 6995
information: 6735
knowledge: 6545
time: 6499
looking: 6489
business: 6452
platform: 6399
remote: 6017
applications: 5967
join: 5901
type: 5871
employment: 5802
working: 5778
function: 5430
industries: 5379
seniority: 5199
application: 4779
technical: 4648
programs: 4494
requirements: 4483
projects: 4455
learning: 4399
design: 4193
position: 4136
management: 3911
opportunities: 3895
career: 3893
new: 3828
strong: 3813
systems: 3644
financial: 3569
including: 3505
set: 3504
engineering: 3498
test: 3410
customer: 3310
solutions: 3266


In [12]:

##### code from copilot to look for curated skills on 'criteria' column
skills_list = [
    'python', 'java', 'javascript', 'c++', 'c#', 'typescript', 'go', 'ruby', 'php', 'swift', 'kotlin', 'r',
    'sql', 'excel', 'tableau', 'powerbi', 'sas', 'spark', 'hadoop',
    'aws', 'azure', 'gcp',
    'docker', 'kubernetes', 'jenkins', 'git',
    'tensorflow', 'pytorch', 'scikit', 'keras',
    'linux', 'bash', 'shell', 'jira', 'agile', 'scrum'
]

filtered_skills = {word: freq for word, freq in word_freq.items() if word in skills_list}
sorted_skills = sorted(filtered_skills.items(), key=lambda x: x[1], reverse=True)

for skill, freq in sorted_skills:
    print(f"{skill}: {freq}")


java: 2992
aws: 2293
sql: 2218
python: 1820
agile: 1488
azure: 1444
javascript: 1415
scrum: 658
git: 628
gcp: 473
excel: 450
spark: 439
jira: 412
docker: 411
jenkins: 409
linux: 403
kubernetes: 357
hadoop: 271
tableau: 267
php: 224
typescript: 217


In [13]:
#Consolidation of new dataframe without the 'criteria' column
#renaming of 'title' to 'job_title'
#renaming of 'company' to 'company_name'
jobs_df = complete_df.drop(columns=['criteria'])
jobs_df.rename(columns={'title':'job_title', 'company':'company_name'}, inplace=True)

Column cleaning process continued

In [14]:
#'industry' column standarization
jobs_df['industry'] = jobs_df['industry'].str.lower().str.strip()
jobs_df['industry'] = jobs_df['industry'].replace('not avilable', 'unknown')
jobs_df['industry'] = jobs_df['industry'].fillna('unknown')
print(jobs_df['industry'].value_counts().head(50))

industry
unknown                                                7569
it services and it consulting                          2498
software development                                    146
information technology & services                       127
technology information and internet                      99
financial services                                       91
staffing and recruiting                                  90
business consulting and services                         73
human resources services                                 44
information services                                     31
advertising services                                     27
non-profit organizations                                 27
retail                                                   22
telecommunications                                       22
motor vehicle manufacturing                              20
outsourcing and offshoring consulting                    15
pharmaceutical manufacturing   

In [15]:
#'level' column standarization
jobs_df['level'] = jobs_df['level'].str.lower().str.strip()
jobs_df['level'] = jobs_df['level'].replace('not avilable', 'unknown')
jobs_df['level'] = jobs_df['level'].str.replace(r'\s+level$', '', regex=True).str.strip()
jobs_df['level'] = jobs_df['level'].fillna('unknown')
jobs_df.rename(columns={'level':'seniority'}, inplace=True)
print(jobs_df['seniority'].value_counts())

seniority
unknown       7681
mid-senior    2932
associate      400
entry          127
executive       34
director        24
internship       7
Name: count, dtype: int64


In [16]:
print(jobs_df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 11205 entries, 0 to 2844
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   job_title      11205 non-null  object        
 1   company_name   11205 non-null  object        
 2   onsite_remote  11205 non-null  object        
 3   industry       11205 non-null  object        
 4   seniority      11205 non-null  object        
 5   country        11205 non-null  object        
 6   posted_date    11205 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(6)
memory usage: 700.3+ KB
None


In [17]:
# Function to extract skills from each row's criteria text
def extract_skills(text):
    if pd.isna(text):
        return []
    text_lower = text.lower()
    return [skill for skill in skills_list if skill in text_lower]

# Apply the function to create a new column with skill lists
jobs_df['skills'] = complete_df['criteria'].apply(extract_skills)

# Convert the list of skills into a string for Power BI
jobs_df['skills_str'] = jobs_df['skills'].apply(lambda x: ';'.join(x))

# Drop the 'skills' list column and keep only the string version
jobs_df.drop(columns=['skills'], inplace=True)